In [18]:
%reset -f
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
#from linearmodels.iv import IV2SLS
import functions as fun
import statsmodels.api as sm
import seaborn as sns

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
df = pd.read_csv('dataset.csv')
Nobs=df['ID'].count()
df['const']=np.ones((Nobs,1))
data = df[df['Market share'] != 0]

data.head(20)

,ID,Year,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,const
8,1,2021,0.010373,Aiways,U5,400,284621.700,201,34,SUV,C,CN,257,1.0
9,1,2022,0.005976,Aiways,U5,400,313681.829,201,34,SUV,C,CN,183,1.0
10,1,2023,0.002860,Aiways,U5,400,264524.000,201,34,SUV,C,CN,177,1.0
21,2,2023,0.000048,Aiways,U6,405,360638.000,214,34,SUV,C,CN,3,1.0
28,3,2019,0.040630,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0
29,3,2020,0.034680,Audi,e-tron,375,890101.410,402,17,SUV,F,DE,491,1.0
30,3,2021,0.010494,Audi,e-tron,375,800035.193,402,17,SUV,F,DE,260,1.0
31,3,2022,0.017570,Audi,e-tron,375,789723.656,402,17,SUV,F,DE,538,1.0
32,3,2023,0.001099,Audi,e-tron,375,673037.728,402,17,SUV,F,DE,68,1.0
41,4,2021,0.003391,Audi,e-tron GT,472,1278896.110,522,17,Sedan,F,DE,84,1.0


In [7]:
# Copy the dataframe
df2 = data.copy()

In [8]:
#Scale for better intepretation

df2['Price'] = df2['Price']/10_000 #(Change in ms(%) for change in pris in 10.000)
df2['HP'] = df2['HP']/10           #(Change in ms(%) for change in HP in 10)
df2['Range'] = df2['Range']/10     #(Change in ms(%) for change in rækkevidde in 10)

In [9]:
# Creating dummy for china
df2['China'] = (df2['Country'] == 'CN').astype(int)

In [10]:
# Take the log of the market share
df2['log_market_share'] = np.log(df2['Market share'])

# OLS

In [11]:
y = df2['log_market_share']
x = df2[['const', 'Range', 'Price', 'HP', 'Chargetime']]
dummies = df2[['China']]
X = pd.concat([x, dummies], axis=1)
print(X.shape)

(334, 6)


In [12]:
OLS_model = sm.OLS(y, X)
OLS_result = OLS_model.fit(cov_type='HC3')
# Print summary of the regression results
print(OLS_result.summary())

                            OLS Regression Results                            
Dep. Variable:       log_market_share   R-squared:                       0.207
Model:                            OLS   Adj. R-squared:                  0.195
Method:                 Least Squares   F-statistic:                     21.01
Date:                Fri, 03 May 2024   Prob (F-statistic):           3.18e-18
Time:                        15:27:59   Log-Likelihood:                -716.37
No. Observations:                 334   AIC:                             1445.
Df Residuals:                     328   BIC:                             1468.
Df Model:                           5                                         
Covariance Type:                  HC3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -6.4240      0.700     -9.171      0.0

# BLP instrument (sum)

In [13]:
fun.BLP(df2, 'HP')
fun.BLP(df2, 'Chargetime')
fun.BLP(df2, 'Range')

,ID,Year,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,const,China,log_market_share,HP_sum,Chargetime_sum,Range_sum
8,1,2021,0.010373,Aiways,U5,40.0,28.462170,20.1,34,SUV,C,CN,257,1.0,1,-4.568514,1358.3,1861,2151.3
9,1,2022,0.005976,Aiways,U5,40.0,31.368183,20.1,34,SUV,C,CN,183,1.0,1,-5.119955,2033.9,2664,3218.0
10,1,2023,0.002860,Aiways,U5,40.0,26.452400,20.1,34,SUV,C,CN,177,1.0,1,-5.856883,2640.3,3241,4177.4
21,2,2023,0.000048,Aiways,U6,40.5,36.063800,21.4,34,SUV,C,CN,3,1.0,1,-9.934421,2639.0,3241,4176.9
28,3,2019,0.040630,Audi,e-tron,37.5,97.970447,40.2,17,SUV,F,DE,222,1.0,0,-3.203259,390.1,570,580.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1154,189,2023,0.000065,Volvo,EX30,47.5,36.824500,26.8,28,SUV,B,SE,4,1.0,0,-9.646739,2633.6,3247,4169.9
1163,190,2021,0.014652,Volvo,XC40,45.7,46.206060,40.2,28,SUV,C,SE,363,1.0,0,-4.223188,1338.2,1867,2145.6
1164,190,2022,0.033310,Volvo,XC40,45.7,41.626340,40.2,28,SUV,C,SE,1020,1.0,0,-3.401883,2013.8,2670,3212.3
1165,190,2023,0.031752,Volvo,XC40,45.7,43.926660,40.2,28,SUV,C,SE,1965,1.0,0,-3.449786,2620.2,3247,4171.7


In [14]:
y = df2['log_market_share']
x = df2[['const', 'Range', 'HP', 'Chargetime']]
dummies = df2[[ 'China']]
X = pd.concat([x, dummies], axis=1)
k = df2['Price']
z=df2[['Range_sum', 'HP_sum', 'Chargetime_sum']] 

In [15]:
BLP_model = IV2SLS(dependent=y, exog=X, endog=k, instruments=z).fit(cov_type='robust')

NameError: name 'IV2SLS' is not defined

In [ ]:
BLP_model.first_stage

,Price
R-squared,0.5822
Partial R-squared,0.0261
Shea's R-squared,0.0261
Partial F-statistic,17.145
P-value (Partial F-stat),0.0007
Partial F-stat Distn,chi2(3)
==========================,===========
const,-1.3281
,(-0.1735)
Range,0.1875


In [ ]:
BLP_model.summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:       log_market_share   R-squared:                     -5.7877
Estimator:                    IV-2SLS   Adj. R-squared:                -5.8912
No. Observations:                 334   F-statistic:                    28.868
Date:                Thu, May 02 2024   P-value (F-stat)                0.0000
Time:                        16:33:11   Distribution:                  chi2(5)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const         -2.8041     2.3134    -1.2121     0.2255     -7.3383      1.7300
Range          0.1995     0.0588     3.3923     0.0007      0.0842      0.3148
HP             0.3270     0.0955     3.4228     0.0006      0.1398      0.5143
Chargetime    -0.0848     0.0305    -2.7760     0.0055     -0.1447     -0.0249
China         -5.4735     1.1202    -4.8862     0.0000     -7.6690     -3.2779
Price         -0.3222     0.0761    -4.2313     0.0000     -0.4714     -0.1729
==============================================================================

Endogenous: Price
Instruments: Range_sum, HP_sum, Chargetime_sum
Robust Covariance (Heteroskedastic)
Debiased: False
"""

# Gandhi Houde instrument (Local Diff)

In [ ]:
fun.create_instrument_localsum(df2, 'HP', 0.5)
fun.create_instrument_localsum(df2, 'Chargetime', 0.5)
fun.create_instrument_localsum(df2, 'Range', 0.5)

,ID,Year,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,...,Sales,const,China,log_market_share,HP_sum,Chargetime_sum,Range_sum,HP_instrument_localsum,Chargetime_instrument_localsum,Range_instrument_localsum
8,1,2021,0.010373,Aiways,U5,40.0,28.462170,20.1,34,SUV,...,257,1.0,1,-4.568514,1358.3,1861,2151.3,740.5,1317,1499.9
9,1,2022,0.005976,Aiways,U5,40.0,31.368183,20.1,34,SUV,...,183,1.0,1,-5.119955,2033.9,2664,3218.0,1060.2,1796,2153.5
10,1,2023,0.002860,Aiways,U5,40.0,26.452400,20.1,34,SUV,...,177,1.0,1,-5.856883,2640.3,3241,4177.4,1501.0,2099,2792.3
21,2,2023,0.000048,Aiways,U6,40.5,36.063800,21.4,34,SUV,...,3,1.0,1,-9.934421,2639.0,3241,4176.9,1456.2,2099,2746.8
28,3,2019,0.040630,Audi,e-tron,37.5,97.970447,40.2,17,SUV,...,222,1.0,0,-3.203259,390.1,570,580.9,310.5,533,470.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1154,189,2023,0.000065,Volvo,EX30,47.5,36.824500,26.8,28,SUV,...,4,1.0,0,-9.646739,2633.6,3247,4169.9,1777.6,1813,2189.4
1163,190,2021,0.014652,Volvo,XC40,45.7,46.206060,40.2,28,SUV,...,363,1.0,0,-4.223188,1338.2,1867,2145.6,1026.5,1138,1169.9
1164,190,2022,0.033310,Volvo,XC40,45.7,41.626340,40.2,28,SUV,...,1020,1.0,0,-3.401883,2013.8,2670,3212.3,1621.7,1483,1702.8
1165,190,2023,0.031752,Volvo,XC40,45.7,43.926660,40.2,28,SUV,...,1965,1.0,0,-3.449786,2620.2,3247,4171.7,1978.6,1813,2206.7


In [ ]:
z=df2[['Range_instrument_localsum', 'HP_instrument_localsum', 'Chargetime_instrument_localsum']] 
GH_model = IV2SLS(dependent=y, exog=X, endog=k, instruments=z).fit(cov_type='robust')
GH_model.first_stage

,Price
R-squared,0.6105
Partial R-squared,0.0919
Shea's R-squared,0.0919
Partial F-statistic,22.829
P-value (Partial F-stat),4.383e-05
Partial F-stat Distn,chi2(3)
================================,===========
const,1.7422
,(0.2392)
Range,0.4165


In [ ]:
GH_model.summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:       log_market_share   R-squared:                     -1.1894
Estimator:                    IV-2SLS   Adj. R-squared:                -1.2228
No. Observations:                 334   F-statistic:                    73.809
Date:                Thu, May 02 2024   P-value (F-stat)                0.0000
Time:                        16:33:11   Distribution:                  chi2(5)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const         -4.6767     1.4901    -3.1386     0.0017     -7.5972     -1.7562
Range          0.1395     0.0231     6.0487     0.0000      0.0943      0.1847
HP             0.1587     0.0585     2.7127     0.0067      0.0440      0.2734
Chargetime    -0.0478     0.0201    -2.3749     0.0176     -0.0872     -0.0083
China         -3.7943     0.6345    -5.9798     0.0000     -5.0380     -2.5507
Price         -0.1722     0.0448    -3.8463     0.0001     -0.2599     -0.0844
==============================================================================

Endogenous: Price
Instruments: Range_instrument_localsum, HP_instrument_localsum, Chargetime_instrument_localsum
Robust Covariance (Heteroskedastic)
Debiased: False
"""

# Logit

In [ ]:
GH_params = GH_model.params
GH_params = np.array(GH_params)
GH_params = np.array([GH_params[0], GH_params[-1], *GH_params[1:-1]])
GH_params

array([-4.67668433, -0.17218025,  0.13951338,  0.15870948, -0.04775668,
       -3.79431677])

In [ ]:
logit_df = df.copy()
#Scale for better intepretation
logit_df['Price'] = logit_df['Price']/10_000 #(Change in ms(%) for change in pris in 10.000)
logit_df['HP'] = logit_df['HP']/10           #(Change in ms(%) for change in HP in 10)
logit_df['Range'] = logit_df['Range']/10     #(Change in ms(%) for change in rækkevidde in 10)
logit_df = fun.straf_0ms(logit_df)

/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price'][i] = 10_000_000


In [17]:
logit_df['China'] = (logit_df['Country'] == 'CN').astype(int)
x = logit_df[['const', 'Price', 'Range', 'HP', 'Chargetime']]
dummies = logit_df[['China']]
X = pd.concat([x, dummies], axis=1)

alpha = GH_params[1]
alpha = np.array(alpha)
beta = GH_params
beta = np.array(beta)
p_j = logit_df['Price']
p_j = np.array(p_j)
columns = X.columns[1:]
X = np.array(X)

NameError: name 'logit_df' is not defined

In [ ]:
logit_df['logit_market_share'] = fun.ccp(alpha, beta, X, p_j)
logit_df2 = logit_df[logit_df['Market share'] != 0]
ccp = logit_df2['logit_market_share']
ccp = np.array(ccp)

share_j: 1177
 choice probability sum: 1.0 
 ccp:[array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[3.85198732e-05]]), array([[2.33551211e-05]]), array([[5.44464141e-05]])]


In [ ]:
probability_ratio = fun.probability_ratio(ccp, logit_df2['Model'])
probability_ratio

Model,U5,U5,U5,U6,e-tron,e-tron,e-tron,e-tron,e-tron,e-tron GT,...,up!,up!,C40,C40,C40,EX30,XC40,XC40,XC40,Free
Model,,,,,,,,,,,,,,,,,,,,,
U5,[[1.0]],[[1.6493116481277739]],[[0.7074822809820299]],[[2.808839779119382]],[[91.88106369254153]],[[19.642727566686897]],[[4.165952019289861]],[[3.4882512186563783]],[[0.46780643955536233]],[[610.446272323077]],...,[[0.5012478833172723]],[[0.3321339369848353]],[[0.007977010361456899]],[[0.005089310011042686]],[[0.003423368233417925]],[[0.008645358486777506]],[[0.006664259949802337]],[[0.0030289668656644083]],[[0.004500962478169808]],[[0.3459409781160227]]
U5,[[0.6063135497376473]],[[1.0]],[[0.42895609315870215]],[[1.7030376171221815]],[[55.70873388109572]],[[11.909651877487471]],[[2.525873156852355]],[[2.1149739787602226]],[[0.28363738295694185]],[[370.1218462963193]],...,[[0.30391338343257746]],[[0.20137730632161555]],[[0.004836569468548925]],[[0.003085717618510636]],[[0.002075634545662721]],[[0.005241797992872565]],[[0.00404063110653909]],[[0.0018365036523587028]],[[0.0027289945373750943]],[[0.2097487024412395]]
U5,[[1.413462961378958]],[[2.331240926399493]],[[1.0]],[[3.9701909922331]],[[129.8704803815084]],[[27.764267875969356]],[[5.888418878148096]],[[4.930513897555803]],[[0.6612270754060688]],[[862.8431958405222]],...,[[0.7084953175385662]],[[0.4694590181450375]],[[0.0112752086884555]],[[0.007193551199583972]],[[0.004838804201097552]],[[0.012219894008903241]],[[0.009419684604046798]],[[0.004281332475860755]],[[0.0063619437534494705]],[[0.48897475939020674]]
U6,[[0.3560188827550415]],[[0.5871860902813262]],[[0.25187705124421067]],[[1.0]],[[32.71139364216344]],[[6.993181922553525]],[[1.4831575835186854]],[[1.2418833016349557]],[[0.16654792595611395]],[[217.3303998544417]],...,[[0.17845371140194474]],[[0.11824595317037442]],[[0.002839966316611275]],[[0.001811890464125465]],[[0.00121878373372055]],[[0.003077910869479344]],[[0.0023726023817177973]],[[0.0010783693994158826]],[[0.001602427632800378]],[[0.12316152052805268]]
e-tron,[[0.010883635428364935]],[[0.0179505066859784]],[[0.007699979218236456]],[[0.030570388132624443]],[[1.0]],[[0.21378428565451407]],[[0.04534070299001156]],[[0.03796485454640571]],[[0.005091434739162]],[[6.643874676568749]],...,[[0.005455399221264798]],[[0.00361482468353048]],[[8.681887258238647e-05]],[[5.539019474211651e-05]],[[3.725869178956641e-05]],[[9.409292991760712e-05]],[[7.253137569350224e-05]],[[3.296617109048864e-05]],[[4.8986834689150156e-05]],[[0.0037650954855467634]]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EX30,[[115.66900337671743]],[[190.77423459655085]],[[81.83377034787816]],[[324.895697895618]],[[10627.791066508975]],[[2272.054721239047]],[[481.87151818648164]],[[403.48254198960325]],[[54.11070463657936]],[[70609.71193464256]],...,[[57.97884310799804]],[[38.417601478621364]],[[0.9226928384354679]],[[0.5886754168524583]],[[0.39597759175096514]],[[1.0]],[[0.7708483066370092]],[[0.3503575786125016]],[[0.520621844085902]],[[40.01464816584716]]
XC40,[[150.05417068547274]],[[247.48609156170335]],[[106.16066694742511]],[[421.47812364412533]],[[13787.13681408342]],[[2947.4731950198766]],[[625.1184753700106]],[[523.4266437580725]],[[70.19630732880363]],[[91600.00914147757]],...,[[75.21433543902192]],[[49.838082470760526]],[[1.1969836743378381]],[[0.7636721930682844]],[[0.5136906812165187]],[[1.2972720980120123]],[[1.0]],[[0.4545091110610485]],[[0.6753881919482008]],[[51.90988657732107]]
XC40,[[330.14557251706634]],[[544.5129383302101]],[[233.57214270049224]],[[927.3260169860785]],[[30334.12637625116]],[[6484.959538300607]],[[1375.3706144870796]],[[1151.6306956666642]],[[154.44422481417547]],[[201536.13406701124]],...,[[165.48476941074856]],[[109.65254877820567]],[[2.633574652757758]],[[1.6802131673125247]],[[1.1302098653584989]],[[2.854226827232438]],[[2.2001759165300485]],[[1.0]],[[1.485972834233205]],[[114.21088227722824]]


In [ ]:
marginal_effects = fun.marginal_effects(ccp, logit_df2['Model'], columns, beta)
marginal_effects

,const,Price,Range,HP,Chargetime,China
Model,,,,,,
U5,[[-6.632105797919287e-06]],[[5.373912337995465e-06]],[[6.1131363296976535e-06]],[[-1.8395558620746057e-06]],[[-0.00014615653957263496]],NaN
U5,[[-4.021135608572075e-06]],[[3.2582758656289696e-06]],[[3.7064773880891567e-06]],[[-1.1153476446751521e-06]],[[-8.861669032565522e-05]],NaN
U5,[[-9.374235901305553e-06]],[[7.595826047453989e-06]],[[8.64069177988774e-06]],[[-2.600144076429994e-06]],[[-0.00020658685524923746]],NaN
U6,[[-2.361154896488458e-06]],[[1.913214266596679e-06]],[[2.1763919662282134e-06]],[[-6.549166227812884e-07]],[[-5.2034487925992514e-05]],NaN
e-tron,[[-7.218142162689885e-08]],[[5.8487702710734876e-08]],[[6.653314713632215e-08]],[[-2.002105535293158e-08]],[[-1.5907144921799514e-06]],NaN
...,...,...,...,...,...,...
EX30,[[-0.0007671290679342732]],[[0.0006215950843697809]],[[0.0007071003867621318]],[[-0.00021277959322196788]],[[-0.01690578126935645]],NaN
XC40,[[-0.0009951751354050943]],[[0.0008063779592143394]],[[0.000917301602240016]],[[-0.0002760330293132049]],[[-0.02193139833583022]],NaN
XC40,[[-0.002189560365647818]],[[0.0017741733654840393]],[[0.0020182248934429094]],[[-0.0006073212232617462]],[[-0.048252934434320834]],NaN


In [ ]:
elasticity = fun.elasticity(ccp, logit_df2['Model'], columns, beta, X)
elasticity

elasticity shape: 
(334, 6)


,const,Price,Range,HP,Chargetime,China
Model,,,,,,
U5,[[-1721736.1429716381]],[[5.580320380211402]],[[3.189937737536149]],[[-1.623664424000624]],[[-3.7941706091348752]],NaN
U5,[[-1721762.2536793128]],[[5.580405007646781]],[[3.189986114014801]],[[-1.6236890474176962]],[[-3.7942281490082452]],NaN
U5,[[-1721708.7206142985]],[[5.580231501587253]],[[3.1898869309053612]],[[-1.6236385636466921]],[[-3.794110178793864]],NaN
U6,[[-1721778.8541258806]],[[5.580458811367335]],[[3.190016870406356]],[[-1.6237047022871285]],[[-3.7942647312259816]],NaN
e-tron,[[-1721801.744742373]],[[5.580533002162626]],[[3.1900592808777177]],[[-1.6237262890324573]],[[-3.7943151750205635]],NaN
...,...,...,...,...,...,...
EX30,[[-1714130.8803965242]],[[5.555670957639882]],[[3.17584712661922]],[[-1.6164923643741802]],[[-3.777410977378846]],NaN
XC40,[[-1711850.331875376]],[[5.548279470017011]],[[3.1716218521370703]],[[-1.6143417180535398]],[[-3.7723853582054505]],NaN
XC40,[[-1699906.0194795495]],[[5.509566749625952]],[[3.14949208909765]],[[-1.6030777649876649]],[[-3.7460638110720135]],NaN


In [ ]:
cross_elasticity = fun.cross_elasticity(ccp, beta, X, np.array(logit_df2['Model']))
cross_elasticity

(334, 334, 5)
Change in : U5 
 [[[ 6.63236128e+01 -2.14961514e-04 -1.22880731e-04  6.25457571e-05
    1.46156601e-04]]

 [[ 6.63236128e+01 -2.14961514e-04 -1.22880731e-04  6.25457571e-05
    1.46156601e-04]]

 [[ 6.63236128e+01 -2.14961514e-04 -1.22880731e-04  6.25457571e-05
    1.46156601e-04]]

 ...

 [[ 6.63236128e+01 -2.14961514e-04 -1.22880731e-04  6.25457571e-05
    1.46156601e-04]]

 [[ 6.63236128e+01 -2.14961514e-04 -1.22880731e-04  6.25457571e-05
    1.46156601e-04]]

 [[ 6.63236128e+01 -2.14961514e-04 -1.22880731e-04  6.25457571e-05
    1.46156601e-04]]]
Change in : U5 
 [[[ 4.02129051e+01 -1.30334079e-04 -7.45042520e-05  3.79223400e-05
    8.86167275e-05]]

 [[ 4.02129051e+01 -1.30334079e-04 -7.45042520e-05  3.79223400e-05
    8.86167275e-05]]

 [[ 4.02129051e+01 -1.30334079e-04 -7.45042520e-05  3.79223400e-05
    8.86167275e-05]]

 ...

 [[ 4.02129051e+01 -1.30334079e-04 -7.45042520e-05  3.79223400e-05
    8.86167275e-05]]

 [[ 4.02129051e+01 -1.30334079e-04 -7.45042520e-05

# Costs

In [16]:
logit_df

NameError: name 'logit_df' is not defined

In [ ]:
logit_df['cost'] = fun.cost(p_j, ccp, alpha)
filtered_df = df[(df['cost'] >= 0.0001) & (df['cost'] <= 1_000_000)]
sorted_df = filtered_df.sort_values(by='cost', ascending=False)
sorted_df

NameError: name 'fun' is not defined